In [ ]:
#pip install geopandas
#pip install cartopy
#pip install geopy
#!pip install pyhigh
#!pip install netCDF4


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs
from geopy.geocoders import Nominatim
import geopandas as gpd


In [ ]:
pd.read_csv("Meteorite_DATA.csv")

In [ ]:
df = pd.read_csv("Meteorite_DATA.csv")
df.info()

In [ ]:
mapper = {"mass (g)":"mass"}
df.rename(columns=mapper, inplace=True)

In [ ]:
df["year"].value_counts().sort_index()

In [ ]:
#Removing known outliers
df = df[(df.reclat != 0.0) & (df.reclong != 0.0)]
df = df.drop(df[(df['year'] <=860)].index)
df = df.drop(df[(df['year'] >=2014)].index)

In [ ]:
df[df.isna().any(axis=1)]

In [ ]:
df[df['nametype']=='Valid']

In [ ]:
#Replace all NaNs of valid nametype meteorites with their mean mass
valid_df = df[df["nametype"]=='Valid']
valid_mass_mean = valid_df["mass"].mean(skipna=True)

valid_df[valid_df['mass'].isna()]

In [ ]:
df.loc[df["nametype"] == "Valid", "mass"] = df.loc[df["nametype"] == "Valid", "mass"].fillna(valid_mass_mean)

In [ ]:
#updating the valid_df
valid_df = df[df['nametype'] == 'Valid']

In [ ]:
#Replace all NaNs of relict nametype meteorites with their mean mass
relict_df = df[df["nametype"]=='Relict']
relict_mass_mean = relict_df["mass"].mean(skipna=True)

relict_df[relict_df['mass'].isna()]

In [ ]:
df.loc[df["nametype"] == "Relict", "mass"] = df.loc[df["nametype"] == "Relict", "mass"].fillna(relict_mass_mean)

In [ ]:
df["nametype"].value_counts()

In [ ]:
#updating the relict_df
relict_df = df[df['nametype'] == 'Relict']

From a look at the data, we can see there are multiple meteorites with reclat=58.58333
and reclong=13.43333. We can understand that this is probably false data as there is a very small chance that multiple meteorites landed exactly at the same spot. Therefore, they will be dropped from the data frame.

In [ ]:
df = df.drop(df[(df["reclat"]==58.58333) & (df["reclong"]==13.43333)].index)

In [ ]:
#Replace all Nans in "year" with the mean value
mean_year = round(df["year"].mean(skipna=True))
df["year"] = df["year"].fillna(mean_year)

In [ ]:
#Replace all Nans in "reclat" with the mean value
mean_reclat = df["reclat"].mean(skipna=True)
df["reclat"] = df["reclat"].fillna(mean_reclat)

#Replace all Nans in "reclong" with the mean value
mean_reclong = df["reclong"].mean(skipna=True)
df["reclong"] = df["reclong"].fillna(mean_reclong)

In [ ]:
#Replace all Nans in "GeoLocation" with mean values of "reclat" and "reclong"
df["GeoLocation"] = df["GeoLocation"].fillna("(" +mean_reclat.astype(str) + ", " +mean_reclong.astype(str) + ")")

In [ ]:
df[df["fall"]=="Fell"].value_counts()

In [ ]:
df_fell = df.groupby('fall').get_group('Fell')
df_fell.info()
df_found = df.groupby('fall').get_group('Found')
df_found.info()

In [ ]:
#Creating a df only for meteorites that fell after 1900 
df_fell = df[(df["fall"] == "Fell") & (df["year"]>=1900)]
df_fell.loc[:, "year"] = df_fell["year"].astype(int).values

fell_each_year = df_fell["year"].value_counts().sort_index()

In [ ]:
#plotting all meteotite discoveries by year and decade
plt.subplot(211)
df.year.hist(bins=np.arange(1950,2015,1),figsize=(8,7),color="coral")
plt.title('Discoveries per year')
plt.xlim(1950,2015)

plt.subplot(212)
df.year.hist(bins=np.arange(1950,2015,10),figsize=(8,7),color="lightgreen")
plt.title('Discoveries per decade')
plt.xlim(1950,2015)

plt.show()

In [ ]:
#plotting recorded meteorite falls and meteorite found by year and decade
plt.figure(figsize=(8, 8))
plt.subplot(221)
plt.hist(df_fell['year'], bins=np.arange(1920, 2014, 1), color="slateblue", edgecolor="white")
plt.title('Recorded meteorites fell per year 1920-2014')
plt.xlim(1920, 2014)

plt.subplot(222)
plt.hist(df_found['year'], bins=np.arange(1920, 2014, 10),  color="mediumaquamarine",edgecolor="white")
plt.title('Recorded meteorites found per year 1920-2014')
plt.xlim(1920, 2014)

plt.subplot(223)
plt.hist(df_fell['year'], bins=np.arange(1920, 2020, 10), color="slateblue",edgecolor="white")
plt.title('Recorded meteorites fell per decade 1920-2020')
plt.xlim(1920, 2014)

plt.subplot(224)
plt.hist(df_found['year'], bins=np.arange(1920, 2020, 10), color="mediumaquamarine",edgecolor="white")
plt.title('Recorded meteorites found per decade 1920-2020')
plt.xlim(1920, 2014)

plt.tight_layout()
plt.show()

In [ ]:
#plotting a heatmap of all meteorite findings on the world map
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(10, 8))
ax.coastlines(linewidth=0.6)
ax.gridlines(draw_labels=True, linewidth=0.1, color='gray', linestyle='--', alpha=0.7)

h = np.histogram2d(df['reclong'], df['reclat'], bins=(np.arange(-180, 182, 2), np.arange(-90, 92, 2)))
X, Y = np.meshgrid(h[1][:-1] + 1.0, h[2][:-1] + 1.0)

data_interp, x, y = np.log10(h[0].T + 0.1), X, Y

im = ax.pcolormesh(x, y, data_interp, cmap='hot_r')

cbar = plt.colorbar(im, label='Log-Scaled Frequency', orientation='vertical', pad=0.08, aspect=20, shrink=0.5)

ax.set_title('Heatmap of meteorites impacts', fontsize=15)

plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

# Plot for df_fell
plt.subplot(2, 1, 1, projection=ccrs.PlateCarree())
plt.title("Heatmap of meteorites fell")
plt.gca().coastlines(linewidth=0.6)
plt.gca().gridlines(draw_labels=True, linewidth=0.1, color='gray', linestyle='--', alpha=0.7)
h = np.histogram2d(df_fell['reclong'], df_fell['reclat'], bins=(np.arange(-180, 182, 2), np.arange(-90, 92, 2)))
X, Y = np.meshgrid(h[1][:-1] + 1.0, h[2][:-1] + 1.0)
data_interp, x, y = np.log10(h[0].T + 0.1), X, Y
im = plt.pcolormesh(x, y, data_interp, cmap='hot_r')
cbar = plt.colorbar(im, label='Log-Scaled Frequency', orientation='vertical', pad=0.08, aspect=20, shrink=0.9)

# Plot for df_found
plt.subplot(2, 1, 2, projection=ccrs.PlateCarree())
plt.title("Heatmap of meteorites found")
plt.gca().coastlines(linewidth=0.6)
plt.gca().gridlines(draw_labels=True, linewidth=0.1, color='gray', linestyle='--', alpha=0.7)
h = np.histogram2d(df_found['reclong'], df_found['reclat'], bins=(np.arange(-180, 182, 2), np.arange(-90, 92, 2)))
X, Y = np.meshgrid(h[1][:-1] + 1.0, h[2][:-1] + 1.0)
data_interp, x, y = np.log10(h[0].T + 0.1), X, Y
im = plt.pcolormesh(x, y, data_interp, cmap='hot_r')
cbar = plt.colorbar(im, label='Log-Scaled Frequency', orientation='vertical', pad=0.08, aspect=20, shrink=0.9)

plt.show()


Contrary to expectations, regions with lower populations, such as the central United States, show a surprisingly high number of reported meteorite falls. This challenges the assumption that higher populations lead to more sightings. It's worth considering that in sparsely populated areas, there might be a higher concentration of recording tools or fewer obstructions for meteorite observation, contributing to this unexpected trend.

In [ ]:
#plotting the heatmap for USA fell and found meteorites
df_usa = df[((df["reclat"] >= 25) & (df["reclat"] <= 49) & (df["reclong"] >= -125) & (df["reclong"] <= -67))]

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(10, 8))
ax.coastlines(linewidth=0.6)
ax.gridlines(draw_labels=True, linewidth=0.1, color='gray', linestyle='--', alpha=0.7)

h = np.histogram2d(df_usa['reclong'], df_usa['reclat'], bins=(np.arange(-180, 182, 2), np.arange(-90, 92, 2)))
X, Y = np.meshgrid(h[1][:-1] + 1.0, h[2][:-1] + 1.0)

data_interp, x, y = np.log10(h[0].T + 0.1), X, Y

im = ax.pcolormesh(x, y, data_interp, cmap='hot_r')

cbar = plt.colorbar(im, label='Log-Scaled Frequency', orientation='vertical', pad=0.09, aspect=20, shrink=0.5)

ax.set_title('Heatmap of fell & found meteorites in USA', fontsize=15)

plt.show()

When we focus on meteorite data from the USA, we notice a lot of meteorite impacts happening all over the country. This suggests that the information in the USA data is quite reliable. As a result, we can use this data as a reference for calculations worldwide.

In [ ]:
df_usa["year"].value_counts().sort_index()

In [ ]:
df_fell_usa = df_usa[(df_usa["fall"] == "Fell") & (df_usa["year"]>= 1900)]
df_fell_usa.loc[:, "year"] = df_fell_usa["year"].astype(int).values

In [ ]:
df_usa_new = df_usa[df_usa["year"]>= 1900]
df_usa_new.loc[:, "year"] = df_usa_new["year"].astype(int).values

num_each_year_usa = df_usa_new["year"].value_counts().sort_index()
plt.figure(figsize=(10, 5)) 
plt.bar(num_each_year_usa.index, num_each_year_usa.values, width=1, color="violet",edgecolor="white")

plt.xlabel('Year')
plt.ylabel('Number of Recorded Meteorites in USA', fontsize = 12)
plt.title('Number of Recorded Meteorites Each Year in USA')
plt.xticks(range(min(num_each_year_usa.index), max(num_each_year_usa.index)+1,10))


plt.show()

Given the assumption of reliable data in the USA, we can estimate the global count of meteorite falls by comparing the land area of the USA to the total worldwide land area.

In [ ]:
df_fell_usa = df_usa_new[(df_usa["fall"] == "Fell")]

falls_per_year_usa = df_fell_usa["year"].value_counts().sort_index()

# Land areas in million square kilometers 
usa_area = 9.8  
world_area = 148.94 

area_ratio = world_area /usa_area 

estimated_falls_worldwide = falls_per_year_usa * area_ratio
estimated_falls_worldwide

In [ ]:
plt.figure(figsize=(9, 6))
estimated_falls_worldwide.plot(kind='bar', color='skyblue', width=0.9)

plt.xlabel('Year')
plt.ylabel('Number of Meteorite Falls Worldwide', fontsize=12)
plt.title('Estimated Meteorite Falls Worldwide Each Year')

plt.xticks(fontsize=8)  

plt.tight_layout()
plt.show()

In [ ]:
labels = df['recclass'].value_counts().nlargest(30).index
counts = df['recclass'].value_counts().nlargest(30)

plt.figure(figsize=(15,8))
plt.pie(counts,
        autopct='%0.1f%%',
        labels = labels,
        pctdistance=0.85,
        colors=sns.color_palette('Set2'),
        labeldistance=1.05,
        radius = 8)
plt.rcParams['font.size'] = 8
plt.axis('equal')
plt.tight_layout()
plt.show()

The pie chart shows that the majority of the population belongs to the L, LL, and H class variants. Those letters designate the most common types of ordinary chondrite meteorites, as seen in the following table. Therefore, all subtypes within L, LL, and H classes will be grouped together, while the remaining outliers will be categorized under the "Others" class.

![Alt text](Ordinary_chondrite_meteorites.png)


In [ ]:
df2 = df.copy()
LL = set(df2[df2.recclass.str.startswith("LL")]['recclass'])
L = set(df2[df2.recclass.str.startswith("L")]['recclass'])
L = L-LL
H = set(df2[df2.recclass.str.startswith("H")]['recclass'])
others = set(df2.recclass.values)
others = others - L - LL - H
LL = list(LL)
L = list(L)
H = list(H)
others = list(others)

In [ ]:
df2.recclass.replace(to_replace=LL, value="LL",inplace=True)
df2.recclass.replace(to_replace=L, value="L",inplace=True)
df2.recclass.replace(to_replace=H, value="H",inplace=True)
df2.recclass.replace(to_replace=others, value="others",inplace=True)

In [ ]:
labels = df2['recclass'].unique()
counts = df2['recclass'].value_counts()
plt.figure(figsize=(15,8))
plt.pie(counts,
        autopct='%0.1f%%',
        labels = labels,
        pctdistance=0.850,
        colors=sns.color_palette('Set2'),
        labeldistance=1.1,
        radius = 8)
plt.rcParams['font.size'] = 15
plt.axis('equal')
plt.tight_layout()
plt.show()

## 

One of NASA's tool to detect meteorite falls is all sky cameras. These are wide angle cameras that continuously monitor the entire sky. Rain can significantly impede the camera's ability to detect and track meteorites in the sky due to visibility obstruction, image distortion and light scattering. For that reason, we can check if the amount of annual precipitaion is effecting the recorded meteorite falls in the US by using the Pearson correlation.

In [ ]:
df_pr = pd.read_csv("Annual_precipitation_USA.csv")
df_pr.info()

In [ ]:
df_fell_usa = df_usa[(df_usa["fall"] == "Fell") & (df_usa["year"]>= 1900)]

falls_per_year = df_fell_usa['year'].value_counts().reset_index()
falls_per_year.columns = ['year', 'meteorite_falls']

merged_data = pd.merge(falls_per_year, df_pr, on='year', how='inner')

corr = merged_data['meteorite_falls'].corr(merged_data['volume (inch)'])

print(f"Pearson correlation coefficient: {corr}")

if corr > 0:
    if corr > 0.5:
        print("There is a strong positive correlation.")
    else:
        print("There is a moderate positive correlation.")
elif corr < 0:
    if corr < -0.5:
        print("There is a strong negative correlation.")
    else:
        print("There is a moderate negative correlation.")
else:
    print("There is no significant linear relationship.")


As expected, we can see there is a negative correlation between the amount of annual precipitation and the amount of recorded meteorite falls. Saying that, the correleation coefficient is closer to 0 rather then to 1, so we can't say the precipitation is effecting the recorded meteorite falls significantly.

In [ ]:
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
sns.pairplot(df[['mass', 'year', 'reclat', 'reclong']])
plt.title('Pairplot of Meteorite Data')
plt.show()


In [ ]:
sns.countplot(data=df, x='nametype')
plt.title('Count of Nametypes')
plt.show()


In [ ]:
sns.boxplot(data=df, x='fall', y='mass')
plt.title('Mass Distribution by Fall Type')
plt.show()
